In [ ]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display
import datetime
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
UserId = widgets.IntText(description="Userid:")
k = widgets.IntText(description="no.of.similar users:")
n = widgets.IntText(description="No.of.recommendations:")
button= widgets.Button(description="Recommendation")

Inputs = widgets.VBox ([UserId, k, n])
Interface= widgets.HBox([Inputs, button])


db=pd.DataFrame(columns={'UserId','k','n','Time'})


In [ ]:
def StoretoDb(response):
  global db
  current_timestamp=datetime.datetime.now()
  response.append(current_timestamp)
  new_row=pd.DataFrame(response, index=['UserId','k','n','Time']).T
  db = db.append(new_row,ignore_index=True)
  return("Stored successfully")

In [ ]:
def on_button_clicked(b):
  response=[UserId.value,k.value,n.value]
  print(StoretoDb(response))

button.on_click(on_button_clicked)

In [ ]:
Interface

Stored successfully


In [ ]:
db

,k,UserId,Time,n
0,100,200,2022-08-26 15:45:50.304927,6


In [ ]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
print(ratings.shape)
ratings.head()

(105339, 4)


,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [ ]:
print('The ratings dataset has', ratings['userId'].nunique(), 'unique users')

print('The ratings dataset has', ratings['movieId'].nunique(), 'unique movies')

print('The ratings dataset has', ratings['rating'].nunique(), 'unique ratings')

print('The unique ratings are', sorted(ratings['rating'].unique()))

The ratings dataset has 668 unique users
The ratings dataset has 10325 unique movies
The ratings dataset has 10 unique ratings
The unique ratings are [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


In [ ]:
df = pd.merge(ratings, movies, on='movieId', how='inner')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,9,16,4.0,842686699,Casino (1995),Crime|Drama
2,12,16,1.5,1144396284,Casino (1995),Crime|Drama
3,24,16,4.0,963468757,Casino (1995),Crime|Drama
4,29,16,3.0,836820223,Casino (1995),Crime|Drama


In [ ]:
agg_ratings = df.groupby('title').agg(mean_rating = ('rating', 'mean'),
                                                number_of_ratings = ('rating', 'count')).reset_index()
agg_ratings

,title,mean_rating,number_of_ratings
0,'71 (2014),3.500000,1
1,'Hellboy': The Seeds of Creation (2004),3.000000,1
2,'Round Midnight (1986),2.500000,1
3,'Til There Was You (1997),4.000000,3
4,"'burbs, The (1989)",3.125000,20
...,...,...,...
10318,loudQUIETloud: A Film About the Pixies (2006),4.500000,1
10319,xXx (2002),2.958333,24
10320,xXx: State of the Union (2005),2.071429,7
10321,¡Three Amigos! (1986),3.012500,40


In [ ]:
#movies with over 100 ratings
agg_ratings_GT100 = agg_ratings[agg_ratings['number_of_ratings']>100]
agg_ratings_GT100.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 60 to 10204
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              150 non-null    object 
 1   mean_rating        150 non-null    float64
 2   number_of_ratings  150 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 4.7+ KB


In [ ]:
# popular movies
agg_ratings_GT100.sort_values(by='number_of_ratings', ascending=False).head()

,title,mean_rating,number_of_ratings
7323,Pulp Fiction (1994),4.160000,325
3349,Forrest Gump (1994),4.138264,311
8136,"Shawshank Redemption, The (1994)",4.454545,308
4934,Jurassic Park (1993),3.659864,294
8228,"Silence of the Lambs, The (1991)",4.194828,290


In [ ]:
# Merge data
df_GT100 = pd.merge(df, agg_ratings_GT100[['title']], on='title', how='inner')
df_GT100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22556 entries, 0 to 22555
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     22556 non-null  int64  
 1   movieId    22556 non-null  int64  
 2   rating     22556 non-null  float64
 3   timestamp  22556 non-null  int64  
 4   title      22556 non-null  object 
 5   genres     22556 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 1.2+ MB


In [ ]:
df_GT100

,userId,movieId,rating,timestamp,title,genres
0,1,32,4.0,1217896246,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
1,2,32,3.0,859046895,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,4,32,5.0,950323750,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,7,32,5.0,1322058768,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
4,8,32,3.0,858610933,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
...,...,...,...,...,...,...
22551,632,3147,4.0,1388950398,"Green Mile, The (1999)",Crime|Drama
22552,650,3147,5.0,965432663,"Green Mile, The (1999)",Crime|Drama
22553,659,3147,4.0,1394645567,"Green Mile, The (1999)",Crime|Drama
22554,667,3147,4.0,1083274228,"Green Mile, The (1999)",Crime|Drama


In [ ]:
print('The ratings dataset has', df_GT100['userId'].nunique(), 'unique users')

print('The ratings dataset has', df_GT100['movieId'].nunique(), 'unique movies')

print('The ratings dataset has', df_GT100['rating'].nunique(), 'unique ratings')

print('The unique ratings are', sorted(df_GT100['rating'].unique()))

The ratings dataset has 653 unique users
The ratings dataset has 150 unique movies
The ratings dataset has 10 unique ratings
The unique ratings are [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


In [ ]:
#  user-item matrix
matrix = df_GT100.pivot_table(index='userId', columns='title', values='rating')
matrix.head()

title,2001: A Space Odyssey (1968),Ace Ventura: Pet Detective (1994),Aladdin (1992),Alien (1979),Aliens (1986),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),American Pie (1999),Apocalypse Now (1979),Apollo 13 (1995),Austin Powers: International Man of Mystery (1997),Austin Powers: The Spy Who Shagged Me (1999),Babe (1995),Back to the Future (1985),Back to the Future Part II (1989),Batman (1989),Batman Begins (2005),Batman Forever (1995),"Beautiful Mind, A (2001)",Beauty and the Beast (1991),Beetlejuice (1988),Being John Malkovich (1999),"Big Lebowski, The (1998)","Birdcage, The (1996)",Blade Runner (1982),"Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Broken Arrow (1996),"Bug's Life, A (1998)",Casablanca (1942),Clear and Present Danger (1994),Clerks (1994),Cliffhanger (1993),"Clockwork Orange, A (1971)",Crimson Tide (1995),"Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)",Dances with Wolves (1990),"Dark Knight, The (2008)",Die Hard (1988),Die Hard: With a Vengeance (1995),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Dumb & Dumber (Dumb and Dumber) (1994),E.T. the Extra-Terrestrial (1982),Edward Scissorhands (1990),Fargo (1996),Ferris Bueller's Day Off (1986),"Fifth Element, The (1997)",Fight Club (1999),Finding Nemo (2003),"Firm, The (1993)","Fish Called Wanda, A (1988)",Forrest Gump (1994),Four Weddings and a Funeral (1994),"Fugitive, The (1993)",Full Metal Jacket (1987),Get Shorty (1995),Ghost (1990),Ghostbusters (a.k.a. Ghost Busters) (1984),Gladiator (2000),"Godfather, The (1972)","Godfather: Part II, The (1974)",GoldenEye (1995),Good Will Hunting (1997),Goodfellas (1990),"Green Mile, The (1999)",Groundhog Day (1993),Heat (1995),Home Alone (1990),"Hunt for Red October, The (1990)",In the Line of Fire (1993),Inception (2010),"Incredibles, The (2004)",Independence Day (a.k.a. ID4) (1996),Indiana Jones and the Last Crusade (1989),Indiana Jones and the Temple of Doom (1984),Interview with the Vampire: The Vampire Chronicles (1994),Jaws (1975),Jerry Maguire (1996),Jurassic Park (1993),Kill Bill: Vol. 1 (2003),L.A. Confidential (1997),"Lion King, The (1994)","Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Return of the King, The (2003)","Lord of the Rings: The Two Towers, The (2002)",Léon: The Professional (a.k.a. The Professional) (Léon) (1994),"Mask, The (1994)","Matrix, The (1999)",Memento (2000),Men in Black (a.k.a. MIB) (1997),Minority Report (2002),Mission: Impossible (1996),"Monsters, Inc. (2001)",Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),Mrs. Doubtfire (1993),Natural Born Killers (1994),"Net, The (1995)",Ocean's Eleven (2001),One Flew Over the Cuckoo's Nest (1975),Outbreak (1995),Pirates of the Caribbean: The Curse of the Black Pearl (2003),Pretty Woman (1990),"Princess Bride, The (1987)",Pulp Fiction (1994),Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Rain Man (1988),Reservoir Dogs (1992),"Rock, The (1996)",Saving Private Ryan (1998),Schindler's List (1993),Seven (a.k.a. Se7en) (1995),Shakespeare in Love (1998),"Shawshank Redemption, The (1994)","Shining, The (1980)",Shrek (2001),"Silence of the Lambs, The (1991)","Sixth Sense, The (1999)",Sleepless in Seattle (1993),Speed (1994),Spider-Man (2002),Stand by Me (1986),Star Trek: Generations (1994),Star Wars: Episode I - The Phantom Menace (1999),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Star Wars: Episode VI - Return of the Jedi (1983),Stargate (1994),Taxi Driver (1976),Terminator 2: Judgment Day (1991),"Terminator, The (1984)",There's Something About Mary (1998),Titanic (1997),Total Recall (1990),Toy Story (1995),Toy Story 2 (1999),Trainspotting (1996),True Lies (1994),"Truman Show, The (1998)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Twister (1996),"Usual Suspects, The (1995)",Waterworld (1

In [ ]:
# Normalize user-item matrix
matrix_norm = matrix.subtract(matrix.mean(axis=1), axis = 'rows')
matrix_norm.head()

title,2001: A Space Odyssey (1968),Ace Ventura: Pet Detective (1994),Aladdin (1992),Alien (1979),Aliens (1986),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),American Pie (1999),Apocalypse Now (1979),Apollo 13 (1995),Austin Powers: International Man of Mystery (1997),Austin Powers: The Spy Who Shagged Me (1999),Babe (1995),Back to the Future (1985),Back to the Future Part II (1989),Batman (1989),Batman Begins (2005),Batman Forever (1995),"Beautiful Mind, A (2001)",Beauty and the Beast (1991),Beetlejuice (1988),Being John Malkovich (1999),"Big Lebowski, The (1998)","Birdcage, The (1996)",Blade Runner (1982),"Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Broken Arrow (1996),"Bug's Life, A (1998)",Casablanca (1942),Clear and Present Danger (1994),Clerks (1994),Cliffhanger (1993),"Clockwork Orange, A (1971)",Crimson Tide (1995),"Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)",Dances with Wolves (1990),"Dark Knight, The (2008)",Die Hard (1988),Die Hard: With a Vengeance (1995),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Dumb & Dumber (Dumb and Dumber) (1994),E.T. the Extra-Terrestrial (1982),Edward Scissorhands (1990),Fargo (1996),Ferris Bueller's Day Off (1986),"Fifth Element, The (1997)",Fight Club (1999),Finding Nemo (2003),"Firm, The (1993)","Fish Called Wanda, A (1988)",Forrest Gump (1994),Four Weddings and a Funeral (1994),"Fugitive, The (1993)",Full Metal Jacket (1987),Get Shorty (1995),Ghost (1990),Ghostbusters (a.k.a. Ghost Busters) (1984),Gladiator (2000),"Godfather, The (1972)","Godfather: Part II, The (1974)",GoldenEye (1995),Good Will Hunting (1997),Goodfellas (1990),"Green Mile, The (1999)",Groundhog Day (1993),Heat (1995),Home Alone (1990),"Hunt for Red October, The (1990)",In the Line of Fire (1993),Inception (2010),"Incredibles, The (2004)",Independence Day (a.k.a. ID4) (1996),Indiana Jones and the Last Crusade (1989),Indiana Jones and the Temple of Doom (1984),Interview with the Vampire: The Vampire Chronicles (1994),Jaws (1975),Jerry Maguire (1996),Jurassic Park (1993),Kill Bill: Vol. 1 (2003),L.A. Confidential (1997),"Lion King, The (1994)","Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Return of the King, The (2003)","Lord of the Rings: The Two Towers, The (2002)",Léon: The Professional (a.k.a. The Professional) (Léon) (1994),"Mask, The (1994)","Matrix, The (1999)",Memento (2000),Men in Black (a.k.a. MIB) (1997),Minority Report (2002),Mission: Impossible (1996),"Monsters, Inc. (2001)",Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),Mrs. Doubtfire (1993),Natural Born Killers (1994),"Net, The (1995)",Ocean's Eleven (2001),One Flew Over the Cuckoo's Nest (1975),Outbreak (1995),Pirates of the Caribbean: The Curse of the Black Pearl (2003),Pretty Woman (1990),"Princess Bride, The (1987)",Pulp Fiction (1994),Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Rain Man (1988),Reservoir Dogs (1992),"Rock, The (1996)",Saving Private Ryan (1998),Schindler's List (1993),Seven (a.k.a. Se7en) (1995),Shakespeare in Love (1998),"Shawshank Redemption, The (1994)","Shining, The (1980)",Shrek (2001),"Silence of the Lambs, The (1991)","Sixth Sense, The (1999)",Sleepless in Seattle (1993),Speed (1994),Spider-Man (2002),Stand by Me (1986),Star Trek: Generations (1994),Star Wars: Episode I - The Phantom Menace (1999),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Star Wars: Episode VI - Return of the Jedi (1983),Stargate (1994),Taxi Driver (1976),Terminator 2: Judgment Day (1991),"Terminator, The (1984)",There's Something About Mary (1998),Titanic (1997),Total Recall (1990),Toy Story (1995),Toy Story 2 (1999),Trainspotting (1996),True Lies (1994),"Truman Show, The (1998)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Twister (1996),"Usual Suspects, The (1995)",Waterworld (1

In [ ]:
# User similarity matrix - Pearson correlation
user_similarity = matrix_norm.T.corr()
user_similarity.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,238,239,240,241,242,243,244,245,246,247,248,250,251,252,253,254,255,...,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-0.306186,0.614559,0.431124,-0.094491,0.471405,0.153463,0.369274,0.195776,-1.0,0.552950,NaN,NaN,0.522233,-0.469457,0.510637,-0.212318,0.086394,0.178795,NaN,0.31046,-0.026760,-0.084108,1.839639e-01,0.702345,0.940582,-0.159199,-0.053983,0.014533,0.414347,0.133462,0.144661,0.496774,1.0,0.147442,0.700877,0.274075,0.302048,0.378454,NaN,-0.097187,-0.134706,-0.662266,-0.158207,0.566820,0.241052,0.656485,NaN,0.000000,0.043853,0.547723,-0.287779,-0.514496,0.578296,-0.439372,0.591892,-0.121781,-0.452895,0.211535,0.514839,0.181501,0.225965,0.009493,0.544345,0.072668,1.0,-1.0,0.300933,-0.154474,0.286305,0.140699,-0.25,6.446479e-01,0.196203,-0.034199,0.651429,NaN,0.453313,-0.175219,NaN,NaN,0.721100,0.521321,0.350489,0.208893,0.293527,0.491171,0.427067,0.5,-0.197344,0.609732,0.245718,0.777714,0.342997,0.872872,0.438529,-0.327327,NaN,-0.362262,0.139378,-0.118297,0.945343,0.264149,0.052026,-0.316228,0.196308,1.475052e-01,0.122713,0.224902,0.007302,0.088303,0.375557,-0.038636,-0.30578,0.670721,1.000000,0.061910,0.010891,NaN,0.816497,0.637487,0.440476,0.757904,NaN,0.228829,0.257795,0.426697,0.648886,0.451990,-0.039299,-0.081117,0.523240,0.861555,-0.746203,NaN,0.164943,0.620174,0.280976,0.150908,0.102646,-0.261785,NaN,0.178647,0.581728,0.580036,0.048603,0.611862,0.249365,0.268224,0.758459,0.185695,1.0,-0.174078,-0.112719,0.312559,0.215939,0.624885,0.357510,0.369280,-0.167680,0.481779,0.422856,0.514496,-0.040962,-0.063628,3.062607e-01,0.231803,0.316699,NaN,-0.359211,-0.375,-4.783065e-17,0.408248,0.258873,0.311962,0.210256,0.036367,0.499362,0.534263,NaN,-0.423276,0.403405

In [ ]:
picked_userid = db['UserId'][0]

# Remove picked user ID from the user list
user_similarity.drop(index=picked_userid, inplace=True)

user_similarity.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,238,239,240,241,242,243,244,245,246,247,248,250,251,252,253,254,255,...,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-0.306186,0.614559,0.431124,-0.094491,0.471405,0.153463,0.369274,0.195776,-1.0,0.552950,NaN,NaN,0.522233,-0.469457,0.510637,-0.212318,0.086394,0.178795,NaN,0.31046,-0.026760,-0.084108,1.839639e-01,0.702345,0.940582,-0.159199,-0.053983,0.014533,0.414347,0.133462,0.144661,0.496774,1.0,0.147442,0.700877,0.274075,0.302048,0.378454,NaN,-0.097187,-0.134706,-0.662266,-0.158207,0.566820,0.241052,0.656485,NaN,0.000000,0.043853,0.547723,-0.287779,-0.514496,0.578296,-0.439372,0.591892,-0.121781,-0.452895,0.211535,0.514839,0.181501,0.225965,0.009493,0.544345,0.072668,1.0,-1.0,0.300933,-0.154474,0.286305,0.140699,-0.25,6.446479e-01,0.196203,-0.034199,0.651429,NaN,0.453313,-0.175219,NaN,NaN,0.721100,0.521321,0.350489,0.208893,0.293527,0.491171,0.427067,0.5,-0.197344,0.609732,0.245718,0.777714,0.342997,0.872872,0.438529,-0.327327,NaN,-0.362262,0.139378,-0.118297,0.945343,0.264149,0.052026,-0.316228,0.196308,1.475052e-01,0.122713,0.224902,0.007302,0.088303,0.375557,-0.038636,-0.30578,0.670721,1.000000,0.061910,0.010891,NaN,0.816497,0.637487,0.440476,0.757904,NaN,0.228829,0.257795,0.426697,0.648886,0.451990,-0.039299,-0.081117,0.523240,0.861555,-0.746203,NaN,0.164943,0.620174,0.280976,0.150908,0.102646,-0.261785,NaN,0.178647,0.581728,0.580036,0.048603,0.611862,0.249365,0.268224,0.758459,0.185695,1.0,-0.174078,-0.112719,0.312559,0.215939,0.624885,0.357510,0.369280,-0.167680,0.481779,0.422856,0.514496,-0.040962,-0.063628,3.062607e-01,0.231803,0.316699,NaN,-0.359211,-0.375,-4.783065e-17,0.408248,0.258873,0.311962,0.210256,0.036367,0.499362,0.534263,NaN,-0.423276,0.403405

In [ ]:
# Number of similar users
k = db['k'][0]
user_similarity_threshold = 0.3
#top k similar users
similar_users = user_similarity[user_similarity[picked_userid]>user_similarity_threshold][picked_userid].sort_values(ascending=False)[:k]
# Print- top k similar users
print(f'The similar users for user {picked_userid} are', similar_users)

The similar users for user 200 are userId
282    1.000000
339    1.000000
383    1.000000
22     1.000000
355    1.000000
576    1.000000
347    1.000000
345    1.000000
315    1.000000
141    1.000000
261    1.000000
617    1.000000
623    1.000000
217    1.000000
634    1.000000
639    1.000000
386    1.000000
388    1.000000
395    1.000000
401    1.000000
407    1.000000
570    1.000000
414    1.000000
418    1.000000
565    1.000000
433    1.000000
447    1.000000
454    1.000000
485    1.000000
538    1.000000
516    1.000000
518    1.000000
522    1.000000
148    1.000000
527    1.000000
41     1.000000
42     1.000000
46     1.000000
52     1.000000
55     1.000000
83     1.000000
91     1.000000
655    1.000000
218    0.981981
140    0.962533
110    0.958706
524    0.944911
667    0.944911
449    0.944911
113    0.944755
202    0.943880
585    0.943880
234    0.943242
484    0.942809
632    0.931695
337    0.928571
544    0.927173
402    0.926844
274    0.923381
554    0.91287

In [ ]:
# Movies that the picked user has watched
picked_userid_watched = matrix_norm[matrix_norm.index == picked_userid].dropna(axis=1, how='all')
picked_userid_watched

title,Forrest Gump (1994),"Fugitive, The (1993)",Independence Day (a.k.a. ID4) (1996),Pulp Fiction (1994),"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)"
userId,,,,,,
200,0.666667,-0.333333,-0.833333,0.166667,0.666667,-0.333333


In [ ]:
# Movies that similar users watched. Remove movies that none of the similar users have watched
similar_user_movies = matrix_norm[matrix_norm.index.isin(similar_users.index)].dropna(axis=1, how='all')
similar_user_movies

title,2001: A Space Odyssey (1968),Ace Ventura: Pet Detective (1994),Aladdin (1992),Alien (1979),Aliens (1986),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),American Pie (1999),Apocalypse Now (1979),Apollo 13 (1995),Austin Powers: International Man of Mystery (1997),Austin Powers: The Spy Who Shagged Me (1999),Babe (1995),Back to the Future (1985),Back to the Future Part II (1989),Batman (1989),Batman Begins (2005),Batman Forever (1995),"Beautiful Mind, A (2001)",Beauty and the Beast (1991),Beetlejuice (1988),Being John Malkovich (1999),"Big Lebowski, The (1998)","Birdcage, The (1996)",Blade Runner (1982),"Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Broken Arrow (1996),"Bug's Life, A (1998)",Casablanca (1942),Clear and Present Danger (1994),Clerks (1994),Cliffhanger (1993),"Clockwork Orange, A (1971)",Crimson Tide (1995),"Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)",Dances with Wolves (1990),"Dark Knight, The (2008)",Die Hard (1988),Die Hard: With a Vengeance (1995),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Dumb & Dumber (Dumb and Dumber) (1994),E.T. the Extra-Terrestrial (1982),Edward Scissorhands (1990),Fargo (1996),Ferris Bueller's Day Off (1986),"Fifth Element, The (1997)",Fight Club (1999),Finding Nemo (2003),"Firm, The (1993)","Fish Called Wanda, A (1988)",Forrest Gump (1994),Four Weddings and a Funeral (1994),"Fugitive, The (1993)",Full Metal Jacket (1987),Get Shorty (1995),Ghost (1990),Ghostbusters (a.k.a. Ghost Busters) (1984),Gladiator (2000),"Godfather, The (1972)","Godfather: Part II, The (1974)",GoldenEye (1995),Good Will Hunting (1997),Goodfellas (1990),"Green Mile, The (1999)",Groundhog Day (1993),Heat (1995),Home Alone (1990),"Hunt for Red October, The (1990)",In the Line of Fire (1993),Inception (2010),"Incredibles, The (2004)",Independence Day (a.k.a. ID4) (1996),Indiana Jones and the Last Crusade (1989),Indiana Jones and the Temple of Doom (1984),Interview with the Vampire: The Vampire Chronicles (1994),Jaws (1975),Jerry Maguire (1996),Jurassic Park (1993),Kill Bill: Vol. 1 (2003),L.A. Confidential (1997),"Lion King, The (1994)","Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Return of the King, The (2003)","Lord of the Rings: The Two Towers, The (2002)",Léon: The Professional (a.k.a. The Professional) (Léon) (1994),"Mask, The (1994)","Matrix, The (1999)",Memento (2000),Men in Black (a.k.a. MIB) (1997),Minority Report (2002),Mission: Impossible (1996),"Monsters, Inc. (2001)",Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),Mrs. Doubtfire (1993),Natural Born Killers (1994),"Net, The (1995)",Ocean's Eleven (2001),One Flew Over the Cuckoo's Nest (1975),Outbreak (1995),Pirates of the Caribbean: The Curse of the Black Pearl (2003),Pretty Woman (1990),"Princess Bride, The (1987)",Pulp Fiction (1994),Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Rain Man (1988),Reservoir Dogs (1992),"Rock, The (1996)",Saving Private Ryan (1998),Schindler's List (1993),Seven (a.k.a. Se7en) (1995),Shakespeare in Love (1998),"Shawshank Redemption, The (1994)","Shining, The (1980)",Shrek (2001),"Silence of the Lambs, The (1991)","Sixth Sense, The (1999)",Sleepless in Seattle (1993),Speed (1994),Spider-Man (2002),Stand by Me (1986),Star Trek: Generations (1994),Star Wars: Episode I - The Phantom Menace (1999),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Star Wars: Episode VI - Return of the Jedi (1983),Stargate (1994),Taxi Driver (1976),Terminator 2: Judgment Day (1991),"Terminator, The (1984)",There's Something About Mary (1998),Titanic (1997),Total Recall (1990),Toy Story (1995),Toy Story 2 (1999),Trainspotting (1996),True Lies (1994),"Truman Show, The (1998)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Twister (1996),"Usual Suspects, The (1995)",Waterworld (1

In [ ]:
# Remove the watched movie from the movie list
similar_user_movies.drop(picked_userid_watched.columns,axis=1, inplace=True, errors='ignore')

similar_user_movies

title,2001: A Space Odyssey (1968),Ace Ventura: Pet Detective (1994),Aladdin (1992),Alien (1979),Aliens (1986),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),American Pie (1999),Apocalypse Now (1979),Apollo 13 (1995),Austin Powers: International Man of Mystery (1997),Austin Powers: The Spy Who Shagged Me (1999),Babe (1995),Back to the Future (1985),Back to the Future Part II (1989),Batman (1989),Batman Begins (2005),Batman Forever (1995),"Beautiful Mind, A (2001)",Beauty and the Beast (1991),Beetlejuice (1988),Being John Malkovich (1999),"Big Lebowski, The (1998)","Birdcage, The (1996)",Blade Runner (1982),"Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Broken Arrow (1996),"Bug's Life, A (1998)",Casablanca (1942),Clear and Present Danger (1994),Clerks (1994),Cliffhanger (1993),"Clockwork Orange, A (1971)",Crimson Tide (1995),"Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)",Dances with Wolves (1990),"Dark Knight, The (2008)",Die Hard (1988),Die Hard: With a Vengeance (1995),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Dumb & Dumber (Dumb and Dumber) (1994),E.T. the Extra-Terrestrial (1982),Edward Scissorhands (1990),Fargo (1996),Ferris Bueller's Day Off (1986),"Fifth Element, The (1997)",Fight Club (1999),Finding Nemo (2003),"Firm, The (1993)","Fish Called Wanda, A (1988)",Four Weddings and a Funeral (1994),Full Metal Jacket (1987),Get Shorty (1995),Ghost (1990),Ghostbusters (a.k.a. Ghost Busters) (1984),Gladiator (2000),"Godfather, The (1972)","Godfather: Part II, The (1974)",GoldenEye (1995),Good Will Hunting (1997),Goodfellas (1990),"Green Mile, The (1999)",Groundhog Day (1993),Heat (1995),Home Alone (1990),"Hunt for Red October, The (1990)",In the Line of Fire (1993),Inception (2010),"Incredibles, The (2004)",Indiana Jones and the Last Crusade (1989),Indiana Jones and the Temple of Doom (1984),Interview with the Vampire: The Vampire Chronicles (1994),Jaws (1975),Jerry Maguire (1996),Jurassic Park (1993),Kill Bill: Vol. 1 (2003),L.A. Confidential (1997),"Lion King, The (1994)","Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Return of the King, The (2003)","Lord of the Rings: The Two Towers, The (2002)",Léon: The Professional (a.k.a. The Professional) (Léon) (1994),"Mask, The (1994)","Matrix, The (1999)",Memento (2000),Men in Black (a.k.a. MIB) (1997),Minority Report (2002),Mission: Impossible (1996),"Monsters, Inc. (2001)",Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),Mrs. Doubtfire (1993),Natural Born Killers (1994),"Net, The (1995)",Ocean's Eleven (2001),One Flew Over the Cuckoo's Nest (1975),Outbreak (1995),Pirates of the Caribbean: The Curse of the Black Pearl (2003),Pretty Woman (1990),"Princess Bride, The (1987)",Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Rain Man (1988),Reservoir Dogs (1992),"Rock, The (1996)",Saving Private Ryan (1998),Schindler's List (1993),Seven (a.k.a. Se7en) (1995),Shakespeare in Love (1998),"Shining, The (1980)",Shrek (2001),"Sixth Sense, The (1999)",Sleepless in Seattle (1993),Speed (1994),Spider-Man (2002),Stand by Me (1986),Star Trek: Generations (1994),Star Wars: Episode I - The Phantom Menace (1999),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back (1980),Star Wars: Episode VI - Return of the Jedi (1983),Stargate (1994),Taxi Driver (1976),Terminator 2: Judgment Day (1991),"Terminator, The (1984)",There's Something About Mary (1998),Titanic (1997),Total Recall (1990),Toy Story (1995),Toy Story 2 (1999),Trainspotting (1996),True Lies (1994),"Truman Show, The (1998)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Twister (1996),"Usual Suspects, The (1995)",Waterworld (1995),While You Were Sleeping (1995),Who Framed Roger Rabbit? (1988),Willy Wonka & the Chocolate Factory (1971),"Wizard of Oz, The (1939)",X-Men (2000)
userId,,,,,,,,,,,,,

In [ ]:
item_score = {}

for i in similar_user_movies.columns:
  movie_rating = similar_user_movies[i]
  total = 0
  count = 0
  for u in similar_users.index:
    if pd.isna(movie_rating[u]) == False:
      score = similar_users[u] * movie_rating[u]
      total += score
      count +=1
  item_score[i] = total / count

item_score = pd.DataFrame(item_score.items(), columns=['movie', 'movie_score'])
    
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)

n = db['n'][0]
ranked_item_score.head(n)

,movie,movie_score
49,Fight Club (1999),0.536378
92,Monty Python and the Holy Grail (1975),0.529707
70,Inception (2010),0.494516
137,"Usual Suspects, The (1995)",0.442473
63,Goodfellas (1990),0.408230
6,American Beauty (1999),0.407925
